In [19]:
import numpy as np
import scipy as sc
from tqdm.notebook import tqdm
from scipy import linalg
from scipy.sparse import csr_matrix
from joblib import Parallel, delayed

In [20]:
import os

import numpy as np
from scipy import sparse

#sn.set()

import pandas as pd

In [21]:

import pickle

def save_pkl(obj, filename ):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL )
    
def load_pkl(filename ):
    with open(filename, 'rb') as f:
        return pickle.load(f)
 

In [22]:
DATA_DIR = '/efs/users/hsteck/public/data_for_ease/movielens20mio/'

In [23]:
pro_dir = os.path.join(DATA_DIR, 'pro_sg')


In [24]:
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [25]:
def load_xtx_binary(I):
    train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))
    X=train_data[:,:I]
    print (X.shape)
    return [X.shape[0] , X]

In [35]:
I = 10000
userCnt , X =load_xtx_binary(I)
#X_ = X.toarray()
U = 116677

(116677, 10000)


In [36]:
X.shape

(116677, 10000)

In [37]:
nonzero = csr_matrix.nonzero(X)

In [38]:
Var = np.ones((U,I)) 
Var[nonzero] = 8
#X = csr_matrix(sc.sparse.csr_matrix.multiply(X,Var))

In [39]:
def ease(X, lam = 200.0):
    print('multiplying matrix')
    G = X.T @ X
    diagIndices = np.diag_indices(G.shape[0])
    G = G + lam * np.eye(G.shape[0])
    print('inverting')
    P = linalg.inv(G)
    print('inverting complete')
    B = P / (-np.diag(P))
    B[diagIndices] = 0.0
    return B

In [40]:
ii = np.diag_indices(I-1)

In [41]:
def lsq(w, item, lam = 1.0):
    xw = X.dot(w)
    diff = xw-X_[:,item]
    loss = np.linalg.norm(diff)**2 + lam * (np.inner(w,w))
    return loss

def dlsq(w, item, lam = 1.0):
    xw = X.dot(w)
    return np.multiply(Var[:,item],(xw - X_[:,item])).T@X + lam * w



def weighted_ls(item,lam=200.0):
    mask = np.ones(I, dtype=bool)
    mask[item] = False
    XD = X[:,mask].T.multiply(Var[:,item])
    XtX = XD@X[:,mask]
    XtX[ii] += lam
    return sc.sparse.linalg.cg(XtX.toarray(), XD@X[:,item].toarray(),rtol=1e-10)[0]
    
    



In [42]:
def exp_cf(item):
    lam = 200.0
    w = np.zeros(I)
    mask = np.ones(I, dtype=bool)
    mask[item] = False
    w[mask] = weighted_ls(item,lam)
    return w

In [43]:
W = Parallel(n_jobs=90)(delayed(exp_cf)(item) for item in tqdm(range(I)))
W = np.array(W)

  0%|          | 0/10000 [00:00<?, ?it/s]

Process LokyProcess-195:
Process LokyProcess-197:
Process LokyProcess-196:
Process LokyProcess-194:
Process LokyProcess-192:
Process LokyProcess-190:
Process LokyProcess-198:
Process LokyProcess-188:
Process LokyProcess-189:
Process LokyProcess-185:
Process LokyProcess-180:
Process LokyProcess-181:
Process LokyProcess-182:
Process LokyProcess-175:
Process LokyProcess-177:
Process LokyProcess-173:
Process LokyProcess-174:
Process LokyProcess-172:
Process LokyProcess-176:
Process LokyProcess-170:
Process LokyProcess-167:
Process LokyProcess-169:
Process LokyProcess-165:
Process LokyProcess-166:
Process LokyProcess-156:
Process LokyProcess-154:
Process LokyProcess-160:
Process LokyProcess-155:
Process LokyProcess-159:
Process LokyProcess-164:
Process LokyProcess-150:
Process LokyProcess-151:
Process LokyProcess-158:
Process LokyProcess-145:
Process LokyProcess-142:
Process LokyProcess-139:
Process LokyProcess-138:
Process LokyProcess-140:
Process LokyProcess-135:
Process LokyProcess-143:


In [ ]:
# #Bn_ = (np.ones((I , I)) - 3).flatten()
# Bn = (np.zeros((I , I))).flatten()
# Bn1 = (np.ones((I , I)) + 2).flatten()
# for i in range(10000):
#     #Bnvec = Bn.flatten()
#     #Bn_vec = Bn_.flatten()
#     dBn_ = der(Bn_).flatten()
#     dBn = der(Bn).flatten()
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-132:
#         break
#     gamma = top / bottom
#     #print(gamma)
#     print(np.linalg.norm(der(Bn)))
#     Bn1 = Bn - gamma * der(Bn)
#     Bn_ = Bn
#     Bn = Bn1

# def temp(W):
#     W = W.reshape(I,I)
#     sums = 0.0
#     for i in range(I):
#         xw = X@W[:,i]
#         sums += np.sum( np.logaddexp(-xw/2,xw/2) - X[:,i]/2 * xw)
#     return sums



# def temp2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return np.sum(np.logaddexp(-S/2,S/2) - X / 2 * S) + lam * np.linalg.norm(W,ord='fro') ** 2


# def der2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return (1/2 * X.T @ (sigmoid(S/2) - (sigmoid(-S/2)) + X) + 2 * lam * W).flatten()
    



# def der(W):
#     W = W.reshape(I,I)
#     der = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / 2 * (np.tanh(np.inner(X[u],W[:,i])) - X[u,i])
#             der[i] = der[i] + scalar * X[u]
#     return der.flatten()

# def Hess(W):
#     hess = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / (2 * np.cosh(np.inner(X[u],W[:,i])) + 2)
#             hess = hess + scalar * np.outer(X[u],X[u])
#     return hess

# def lsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     loss = sc.linalg.norm(X - S, ord='fro') ** 2 + lam * (sc.linalg.norm(B, ord='fro') ** 2)
#     return loss

# def dlsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     return (2 * lam * B - 2 * X.T @ (X - S)).flatten()


# def llog(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-Svec/2,Svec/2) - Xvec/2*Svec)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss


# def llog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-S/2,S/2) - X/2*S)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss

# def dllog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     return (1/2 * X.T @ (2*sigmoid(S) - np.ones((U,I)) + X) + 2 * lam * B).flatten
    
    
     

In [ ]:
# Bn = np.zeros(I * I)  
# Bn_ = np.zeros(I * I) + 10 
# Bn1 = np.zeros(I * I) + 100
# gamma = 1.0
# lam = 0.0
# while gamma > 1e-10:
#     dBn_ = der2(Bn_)
#     dBn = der2(Bn)
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-32:
#         break
#     gamma = top / bottom
#     print(temp2(Bn1),np.linalg.norm(der2(Bn1)),gamma)
#     Bn1 = Bn - gamma * der2(Bn)
#     #Bn1mat = Bn1.reshape(I,I)
#     #Bn1 = (Bn1mat - np.diag(np.diag(Bn1mat))).flatten()
#     Bn_ = Bn
#     Bn = Bn1

In [ ]:
# print(temp2(Bn1),np.linalg.norm(der2(np.eye(I) * 100)))

In [ ]:
# W = ease(X)

In [ ]:
# np.linalg.norm(W - Bn1.reshape(I,I))

In [ ]:
# U = 200_000
# I = 20_000
# lam = 100
# #X = np.zeros((U,I))

# p = np.random.uniform(size=I)
# print('sampling matrix')
# X_ = np.random.binomial(n=1,p=p,size=(U,I)) 
# X = csr_matrix(X_)
# X_ = []

In [ ]:
mask = np.ones(I, dtype=bool)
mask[0] = False
w = np.zeros(I-1)
item = 0

In [ ]:
hess = (np.diag(Var[:,item]**2)@X[:,mask]).T @ X[:,mask]
print('1')
diff=(X[:,mask]@w).reshape(U,1) - X[:,item]
print('2')
grad = X[:,mask].T @ (Var[:,item]**2*diff).T

In [ ]:
w = w-sc.sparse.linalg.spsolve(hess,grad)

In [ ]:
w

In [ ]:
np.multiply(np.multiply(Var[:,item],diff),Var[:,item])

In [ ]:
hess.toar

In [ ]:
X.T.shape